In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
np.set_printoptions(precision=5)


### dataset

In [20]:
# data = torch.load("/home/jiazhaohe/datasets/imagenet12/meta.bin")
# print(type(data))
# print(type(data[0]))
# print(type(data[1]))
# print(len(data[0]))
# print(len(data[1]))
# wnid_to_idx = {key:i+1 for i,(key,val) in enumerate(data[0].items())}
# idx_to_wnid = {i+1:key for i,(key,val) in enumerate(data[0].items())}

# val_sample_list = list([])
# val_root = "/home/jiazhaohe/datasets/imagenet12/val"
# for i,j,k in os.walk(val_root):
#     for i,filename in enumerate(sorted(k)):
# #         print(filename)
#         val_sample_list.append((filename,int(lines[i].strip('\n'))))
    
# train_sample_list = list([])
# train_root = "/home/jiazhaohe/datasets/imagenet12/train"
# wnids = os.listdir(train_root)
# for i,wnid in enumerate(wnids):
#     print("loop:",i)
#     imgs = os.listdir(os.path.join(train_root,wnid))
#     for img in imgs:
#         path = wnid + "/" + img
#         train_sample_list.append((path,wnid_to_idx[wnid]))
        

# meta_data = {
#     "wnid_to_class":data[0],
#     "wnid_to_idx":wnid_to_idx,
#     "idx_to_wnid":idx_to_wnid,
#     "val_samples":val_sample_list,
#     "train_samples":train_sample_list
# }

# torch.save(meta_data,"/home/jiazhaohe/datasets/imagenet12/meta_data.bin")

In [2]:

meta = torch.load("/home/jiazhaohe/datasets/imagenet12/meta_data.bin")


In [3]:


import os
from PIL import Image


def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


class ImgListDataSet(torch.utils.data.Dataset):
    
    def __init__(self,root,transform,samples,loader):
        self.root = root
        self.transform = transform
        self.samples = samples
        self.loader = loader
        
    def __getitem__(self,index):
        path,target = self.samples[index]
        sample = self.loader(os.path.join(self.root,path))
        if self.transform is not None:
            sample = self.transform(sample)
        return sample,target
    
    def __len__(self):
        return len(self.samples)
    

In [4]:


ROOT = "/home/jiazhaohe/datasets/imagenet12"
transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])


trainset = ImgListDataSet(os.path.join(ROOT,"train"),
                          transform,meta["train_samples"],pil_loader);
trainloader = torch.utils.data.DataLoader(trainset,batch_size=512,
                                          shuffle=True,num_workers=2)


valset = ImgListDataSet(os.path.join(ROOT,"val"),
                          transform,meta["val_samples"],pil_loader);
valloader = torch.utils.data.DataLoader(valset,batch_size=128,
                                        shuffle=False,num_workers=1)







<class 'torchvision.models.alexnet.AlexNet'>


In [5]:
# net = models.alexnet()   
# net = net.cuda()


net = torch.load("./model.pth")


In [6]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9,weight_decay=1e-5)


In [ ]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i,(inputs,target) in enumerate(trainloader): 
        
#         print(inputs.shape)
#         print(target)
        target = target-1
#         print(target)
        inputs = inputs.cuda()
        target = target.cuda()
    
        outputs = net(inputs)
#         print(outputs)
        loss = criterion(outputs, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0
            


print('Finished Training')

[1,    20] loss: 4.29975
[1,    40] loss: 4.24585
[1,    60] loss: 4.28865
[1,    80] loss: 4.26562
[1,   100] loss: 4.26655
[1,   120] loss: 4.28148
[1,   140] loss: 4.27043
[1,   160] loss: 4.31542
[1,   180] loss: 4.31769
[1,   200] loss: 4.26868
[1,   220] loss: 4.23835
[1,   240] loss: 4.27590
[1,   260] loss: 4.20539
[1,   280] loss: 4.28935
[1,   300] loss: 4.28312
[1,   320] loss: 4.27345
[1,   340] loss: 4.23160
[1,   360] loss: 4.23783
[1,   380] loss: 4.28669
[1,   400] loss: 4.29801
[1,   420] loss: 4.26796
[1,   440] loss: 4.24637
[1,   460] loss: 4.29047
[1,   480] loss: 4.27266
[1,   500] loss: 4.25277
[1,   520] loss: 4.26396
[1,   540] loss: 4.26811
[1,   560] loss: 4.29117
[1,   580] loss: 4.22960
[1,   600] loss: 4.26844
[1,   620] loss: 4.27181
[1,   640] loss: 4.28419
[1,   660] loss: 4.20958
[1,   680] loss: 4.25021
[1,   700] loss: 4.21415
[1,   720] loss: 4.23701
[1,   740] loss: 4.23939
[1,   760] loss: 4.20718
[1,   780] loss: 4.25024


/home/jiazhaohe/anaconda3/envs/python3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


[1,   800] loss: 4.27333
[1,   820] loss: 4.22610
[1,   840] loss: 4.24706
[1,   860] loss: 4.23566
[1,   880] loss: 4.21492
[1,   900] loss: 4.26135
[1,   920] loss: 4.24749
[1,   940] loss: 4.21554


In [8]:
torch.save(net,"./model.pth")

/home/jiazhaohe/anaconda3/envs/python3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type AlexNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/jiazhaohe/anaconda3/envs/python3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/jiazhaohe/anaconda3/envs/python3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/jiazhaohe/anaconda3/envs/python3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked

### evaluation

cuda


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [17]:
top1 = AverageMeter()
top5 = AverageMeter()

net.eval()

for i,(img,label) in enumerate(valloader):
    img,label = img.cuda(),label.cuda()
    out = net(img)
    print(out.shape)
    values,pred_idx = out.topk(5,1,True,True)
    pred_idx = pred_idx.t()+1
#     print(pred_idx[0,:])
#     print(label)
    correct = pred_idx.eq(label.view(1,-1).expand_as(pred_idx))
    res = []
    for k in list([1,5]):
        correct_k = correct[:k].view(-1).float().sum(0)
        
        res.append(correct_k.mul_(100.0/128))
    print("iter:",i,":",res[0],"++",res[1])

#     top1.update(res[0][0],128)
#     top5.update(res[1][0],128)
    if i==1:
        break

RuntimeError: CUDA error: device-side assert triggered

In [54]:
x = torch.Tensor([2,3,5])
print(x)
print(x+1)

tensor([2., 3., 5.])
tensor([3., 4., 6.])


In [15]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

### train

In [ ]:
net = models.alexnet()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data
        
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')